In [ ]:
#default_exp functional

# Functional

> Test function output values against Wolfram | Alpha

In [ ]:
#export
import wolframalpha
from fastcore.test import *
from torch import nn
import torch
import torch.nn.functional as F
import re
import os
import json
import numpy as np
import pickle

In [ ]:
#export
class WolframTester():
    
    def __init__(self, api_key, libdl, cache_path = 'cache.pkl'):
        self.key = api_key
        self.libdl = libdl
        self.cache_path = cache_path
        self.cache = dict()
        self.load_cache(cache_path)
        
    def query(self, expr):
        client = wolframalpha.Client(self.key)
        res = client.query(expr)
        vals = list()
        for pod in res.pods:
            if pod['@title'] == 'Result':
                val = float(pod['subpod']['plaintext'].split('...')[0])
                vals.append(val)
        
        vals = np.around(np.array(val), 16)
        return vals
    
    
    def test(self, fn, fn_expr, xs, shape):
        
        if (fn_expr in self.cache):
            self.test_cache(fn, fn_expr, xs, shape)
            return
    
        if (self.libdl == 'torch'):
            ys = fn(xs).cpu().numpy()
            test_eq(ys.shape, shape)
            _xs = xs.cpu().detach().numpy().flatten()
        
        reals = list()

        for x in _xs:
            expr = re.sub('x', str(x), fn_expr)
            res = self.query(expr)
            reals.append(res)
        
        reals = np.array(reals).reshape(shape)
        np.testing.assert_allclose(ys, reals, rtol=1e-2, atol=1e-5)
        
        self.cache[fn_expr] = (xs, reals)
        self.save_cache(self.cache_path)
        
    
    def test_cache(self, fn, fn_expr, xs, shape):
        xs, reals = self.cache[fn_expr]        
        if (self.libdl == 'torch'):
            ys = fn(xs).cpu().numpy()
            ys = np.around(ys, 16)
            test_eq(ys.shape, shape)
            xs = xs.cpu().detach().numpy().flatten()
        
        np.testing.assert_allclose(ys, reals, rtol=1e-2, atol=1e-5)
        
        
    def save_cache(self, path):
        with open(path, 'wb') as f:
            pickle.dump(self.cache, f, pickle.HIGHEST_PROTOCOL)
            print("Stored Cache")

    def load_cache(self, path):
        if not os.path.exists(path):
            print("No cache found. Will initialize on next query.")
            return
        with open(path, 'rb') as f:
            self.cache = pickle.load(f)
            print("Loaded Cache")

In [ ]:
function = torch.tanh
xs = torch.tensor([[[[-10, -8, -6, -4, -2], [0, 2, 4, 6, 8]]]], dtype=torch.float32)
shape = (1, 1, 2, 5)

In [ ]:
tester = WolframTester('QYU645-4EGHX3JVLE', 'torch')

Loaded Cache


The first time the test function is executed, it queries the Wolfram API. This function call will be slow. 

In [ ]:
%%time
tester.test(function, 'tanh(x)', xs, shape)

Stored Cache
CPU times: user 255 ms, sys: 46.2 ms, total: 302 ms
Wall time: 31.1 s


But now, the inputs and expected outputs are cached on disk in `.pkl` files.

In [ ]:
! ls *.pkl

cache.pkl


So subsequent function calls are much faster!

In [ ]:
%%time
tester.test(function, 'tanh(x)', xs, shape)

CPU times: user 928 µs, sys: 794 µs, total: 1.72 ms
Wall time: 2.53 ms


## Piecewise Functions

In [ ]:
tester.test(F.relu, 'Piecewise[{{0, t < 0 }, {x, t > 0}}] at t = x', xs, shape)

Stored Cache


In [ ]:
tester.test(F.relu, 'Piecewise[{{0, t < 0 }, {x + 2, t > 0}}] at t = x', xs, shape)

AssertionError: 
Not equal to tolerance rtol=0.01, atol=1e-05

Mismatched elements: 4 / 10 (40%)
Max absolute difference: 2.
Max relative difference: 0.5
 x: array([[[[0., 0., 0., 0., 0.],
         [0., 2., 4., 6., 8.]]]], dtype=float32)
 y: array([[[[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  4.,  6.,  8., 10.]]]])

## Mish

In [ ]:
from echoAI.Activation.t_ops import *

In [ ]:
TEST_MISH = [
    Mish(),
    'x * tanh(log(1 + e^x))',
    torch.tensor([[[[-10, -8, -6, -4, -2], [0, 2, 4, 6, 8]]]], dtype=torch.float32),
    (1, 1, 2, 5),
]

In [ ]:
tester.test(*TEST_MISH)

In [ ]:
tester.cache.keys()

dict_keys(['x * tanh(log(1 + e^x))', 'tanh(x)', 'Piecewise[{{0, t < 0 }, {x, t > 0}}] at t = x'])